In [ ]:
%pip install -r "requirements.txt"

In [ ]:
%pip install -q ffmpeg

In [ ]:
%pip install "numpy==1.26.4"

In [ ]:
%pip install "pandas==2.2.2"

In [ ]:
%%bash
set -e
wget -q -O model-bin.zip "https://huggingface.co/MCoRecChallenge/MCoRec-baseline/resolve/main/model-bin.zip"
unzip -o -q model-bin.zip

In [ ]:
%%bash

mkdir -p data-bin
cd data-bin

# Export your Hugging Face token (replace 'your_actual_token_here' with your actual token)
export HF_TOKEN=hf_pCo...~"

# Download the development set
wget --header="Authorization: Bearer $HF_TOKEN" https://huggingface.co/datasets/MCoRecChallenge/MCoRec/resolve/main/dev_without_central_videos.zip

# Unzip the downloaded files
unzip dev_without_central_videos.zip

In [ ]:
%%bash
python script/inference.py --model_type avsr_cocktail --session_dir data-bin/dev/session_132
# For all dev sessions: 
python script/inference.py --model_type avsr_cocktail --session_dir "data-bin/dev/session_132" \
    --checkpoint_path ./model-bin/avsr_cocktail \
    --beam_size 3 --max_length 15 \
    --output_dir_name output_avsr_cocktail

# BL2: MuAViC-EN with AV-HuBERT Transformer decoder
python script/inference.py --model_type muavic_en --session_dir "data-bin/dev/session_132" \
    --checkpoint_path nguyenvulebinh/AV-HuBERT-MuAViC-en \
    --max_length 15 \
    --output_dir_name output_muavic_en


In [ ]:
%%bash
python script/asd.py --video data-bin/dev/session_132/speakers/spk_4/central_crops/track_00.mp4

In [ ]:
%%bash

# BL3: Auto-AVSR with Conformer CTC/Attention
python script/inference.py --model_type auto_avsr --session_dir "data-bin/dev/session_132" \
    --checkpoint_path ./model-bin/auto_avsr/avsr_trlrwlrs2lrs3vox2avsp_base.pth \
    --beam_size 3 --max_length 15 \
    --output_dir_name output_auto_avsr

# BL4: AV-HuBERT CTC/Attention (MCoRec fine-tuned)
python script/inference.py --model_type avsr_cocktail --session_dir "data-bin/dev/session_132" \
    --checkpoint_path ./model-bin/avsr_cocktail_mcorec_finetune \
    --beam_size 3 --max_length 15 \
    --output_dir_name output_avsr_cocktail_finetuned